# Turbulent Flow Example

This notebook demonstrates turbulent flow simulations in PHASTA, including RANS, LES, and DNS approaches.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from phasta import FlowSolver, FlowConfig, Mesh, TurbulenceConfig

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## Problem Setup

We'll simulate turbulent flow in a channel with the following parameters:
- Channel dimensions: 2π × 2 × π
- Reynolds number: 180
- Wall shear stress: 1.0
- Grid resolution: 128 × 129 × 128

In [ ]:
# Create base configuration
config = FlowConfig()
config.domain = {
    'length': 2 * np.pi,
    'height': 2.0,
    'width': np.pi,
    'mesh_size': 0.05
}

config.flow = {
    'reynolds_number': 180,
    'wall_shear_stress': 1.0,
    'fluid_density': 1.0,
    'fluid_viscosity': 1.0/180,
    'time_step': 0.001,
    'max_time': 10.0
}

config.boundary_conditions = {
    'x_periodic': True,
    'y_walls': 'no-slip',
    'z_periodic': True
}

## Direct Numerical Simulation (DNS)

Let's perform a DNS of turbulent channel flow.

In [ ]:
# Create mesh
mesh = Mesh.generate_channel_3d(
    length=config.domain['length'],
    height=config.domain['height'],
    width=config.domain['width'],
    nx=128,
    ny=129,
    nz=128
)

# Plot mesh
plt.figure(figsize=(12, 4))
mesh.plot_slice('y', 0.5)
plt.title('Channel Mesh (y-slice)')
plt.show()

In [ ]:
# Configure DNS
turbulence_config = TurbulenceConfig()
turbulence_config.method = 'dns'
turbulence_config.initial_conditions = {
    'velocity': [1.0, 0.0, 0.0],
    'perturbation': 0.1
}

# Create and run solver
solver = FlowSolver(config, mesh, turbulence_config=turbulence_config)
results = solver.solve()

print("\nDNS Results:")
print(f"Simulation completed: {results.completed}")
print(f"Final time: {results.final_time}")
print(f"Number of time steps: {results.time_steps}")

## Visualize DNS Results

Plot velocity field, vorticity, and turbulent statistics.

In [ ]:
# Plot instantaneous fields
plt.figure(figsize=(15, 5))

# Plot velocity magnitude
plt.subplot(131)
results.plot_velocity_magnitude()
plt.title('Velocity Magnitude')
plt.colorbar(label='Velocity (m/s)')

# Plot vorticity magnitude
plt.subplot(132)
results.plot_vorticity_magnitude()
plt.title('Vorticity Magnitude')
plt.colorbar(label='Vorticity (1/s)')

# Plot pressure field
plt.subplot(133)
results.plot_pressure()
plt.title('Pressure Field')
plt.colorbar(label='Pressure (Pa)')

plt.tight_layout()
plt.show()

In [ ]:
# Plot turbulent statistics
plt.figure(figsize=(15, 5))

# Plot mean velocity profile
plt.subplot(131)
results.plot_mean_velocity_profile()
plt.title('Mean Velocity Profile')
plt.xlabel('y+')
plt.ylabel('u+')
plt.grid(True)

# Plot Reynolds stresses
plt.subplot(132)
results.plot_reynolds_stresses()
plt.title('Reynolds Stresses')
plt.xlabel('y+')
plt.ylabel('Stress')
plt.grid(True)

# Plot turbulent kinetic energy
plt.subplot(133)
results.plot_turbulent_kinetic_energy()
plt.title('Turbulent Kinetic Energy')
plt.xlabel('y+')
plt.ylabel('k+')
plt.grid(True)

plt.tight_layout()
plt.show()

## Large Eddy Simulation (LES)

Now let's perform a LES of the same flow.

In [ ]:
# Configure LES
turbulence_config = TurbulenceConfig()
turbulence_config.method = 'les'
turbulence_config.les = {
    'model': 'smagorinsky',
    'smagorinsky_constant': 0.1,
    'wall_model': 'wall_modeled'
}
turbulence_config.initial_conditions = {
    'velocity': [1.0, 0.0, 0.0],
    'perturbation': 0.1
}

# Create and run solver
solver = FlowSolver(config, mesh, turbulence_config=turbulence_config)
results = solver.solve()

print("\nLES Results:")
print(f"Simulation completed: {results.completed}")
print(f"Final time: {results.final_time}")
print(f"Number of time steps: {results.time_steps}")

## Visualize LES Results

Plot filtered velocity field, subgrid stresses, and turbulent statistics.

In [ ]:
# Plot filtered fields
plt.figure(figsize=(15, 5))

# Plot filtered velocity magnitude
plt.subplot(131)
results.plot_filtered_velocity_magnitude()
plt.title('Filtered Velocity Magnitude')
plt.colorbar(label='Velocity (m/s)')

# Plot subgrid stresses
plt.subplot(132)
results.plot_subgrid_stresses()
plt.title('Subgrid Stresses')
plt.colorbar(label='Stress (Pa)')

# Plot eddy viscosity
plt.subplot(133)
results.plot_eddy_viscosity()
plt.title('Eddy Viscosity')
plt.colorbar(label='Viscosity (m²/s)')

plt.tight_layout()
plt.show()

In [ ]:
# Plot turbulent statistics
plt.figure(figsize=(15, 5))

# Plot mean velocity profile
plt.subplot(131)
results.plot_mean_velocity_profile()
plt.title('Mean Velocity Profile')
plt.xlabel('y+')
plt.ylabel('u+')
plt.grid(True)

# Plot resolved Reynolds stresses
plt.subplot(132)
results.plot_resolved_reynolds_stresses()
plt.title('Resolved Reynolds Stresses')
plt.xlabel('y+')
plt.ylabel('Stress')
plt.grid(True)

# Plot resolved turbulent kinetic energy
plt.subplot(133)
results.plot_resolved_turbulent_kinetic_energy()
plt.title('Resolved Turbulent Kinetic Energy')
plt.xlabel('y+')
plt.ylabel('k+')
plt.grid(True)

plt.tight_layout()
plt.show()

## Compare DNS and LES Results

Compare mean velocity profiles, Reynolds stresses, and turbulent kinetic energy.

In [ ]:
# Plot comparison
plt.figure(figsize=(15, 5))

# Plot mean velocity profiles
plt.subplot(131)
dns_results.plot_mean_velocity_profile(label='DNS')
les_results.plot_mean_velocity_profile(label='LES')
plt.title('Mean Velocity Profile')
plt.xlabel('y+')
plt.ylabel('u+')
plt.legend()
plt.grid(True)

# Plot Reynolds stresses
plt.subplot(132)
dns_results.plot_reynolds_stresses(label='DNS')
les_results.plot_resolved_reynolds_stresses(label='LES')
plt.title('Reynolds Stresses')
plt.xlabel('y+')
plt.ylabel('Stress')
plt.legend()
plt.grid(True)

# Plot turbulent kinetic energy
plt.subplot(133)
dns_results.plot_turbulent_kinetic_energy(label='DNS')
les_results.plot_resolved_turbulent_kinetic_energy(label='LES')
plt.title('Turbulent Kinetic Energy')
plt.xlabel('y+')
plt.ylabel('k+')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Exercises

1. Try different Reynolds numbers
2. Experiment with different LES models
3. Study the effect of grid resolution
4. Compare different wall models

## Next Steps

- Try the conjugate heat transfer example
- Explore multi-phase flows
- Learn about chemical reactions